In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import csv
import json 
import pickle
import gc
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing


In [3]:
tf.enable_eager_execution()

In [4]:
file = pd.read_csv("process/process_file_progress_0.5.csv", header=None)
file.head()

,0,1,2,3,4,5
0,TIF,37,0.0000,0.000000,78.269997,78.279999
1,CMG,38,0.4939,0.401648,400.500000,402.940002
2,DIS,38,0.2960,0.219123,109.559998,109.000000
3,EBAY,38,-0.4215,0.328668,32.450001,33.250000
4,FB,38,-0.2732,0.200821,132.600006,134.199997


In [5]:
file.columns = ['comp','daynum','sent_score','sent_magn','open','close']

In [6]:
company_group = file.groupby(['comp'], as_index=False).apply(pd.DataFrame.sort_values, ['daynum']).reset_index()
company_group = company_group.drop(columns = ['level_0','level_1'])
company_group.head()

,comp,daynum,sent_score,sent_magn,open,close
0,A,74,0.0000,0.000000,53.430000,53.049999
1,A,80,0.3182,0.237337,52.790001,53.130001
2,A,142,0.0000,0.000000,59.799999,58.660000
3,A,198,0.7003,0.693686,61.430000,61.209999
4,A,200,0.2732,0.200821,61.450001,60.980000


In [32]:
comp_data_array = np.array(company_group)
oh_encoder = OneHotEncoder()
comp_encode = oh_encoder.fit_transform(comp_data_array[:,0:1]).toarray()
comp_encode = np.append(comp_encode, comp_data_array[:,1:],1)
comp_encode = preprocessing.normalize(comp_encode)

In [34]:
np.random.shuffle(comp_encode)

In [35]:
comp_encode_train = comp_encode[:19000,:]
comp_encode_test = comp_encode[19000:, :]

In [37]:
comp_encode_train.shape

(19000, 450)

In [15]:
type(comp_encode)

numpy.ndarray

In [72]:
ds = tf.data.Dataset.from_tensor_slices(comp_encode_train)
ds = ds.window(3, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda window: window.batch(3))
ds_train = ds.shuffle(5).map(lambda window: (window[:,:-1], window[:,-1]))
#ds_train= ds.batch(2).prefetch(1)

In [73]:
ds_test = tf.data.Dataset.from_tensor_slices(comp_encode_test)
ds_test = ds_test.window(3, shift=1, drop_remainder=True)
ds_test = ds_test.flat_map(lambda window: window.batch(3))
ds_test = ds_test.shuffle(5).map(lambda window: (window[:,:-1], window[:,-1]))
#ds_test= ds_test.batch(2).prefetch(1)

In [47]:
ds_train.numpy().shape

AttributeError: 'DatasetV1Adapter' object has no attribute 'numpy'

In [74]:
for i,j in ds_train:
    print(i.numpy())
    print(j.numpy())
    print(i.shape)
    print(j.shape)
    break

[[0.         0.         0.         ... 0.         0.         0.13248705]
 [0.         0.         0.         ... 0.0016782  0.00203607 0.10338483]
 [0.         0.         0.         ... 0.         0.         0.12779514]]
[0.13390683 0.10346743 0.12899172]
(3, 449)
(3,)


In [24]:
day_sorted = file.sort_values(['daynum'])
day_sorted.head()

,comp,daynum,sent_score,sent_magn,open,close
0,TIF,37,0.0000,0.000000,78.269997,78.279999
1,CMG,38,0.4939,0.401648,400.500000,402.940002
2,DIS,38,0.2960,0.219123,109.559998,109.000000
3,EBAY,38,-0.4215,0.328668,32.450001,33.250000
4,FB,38,-0.2732,0.200821,132.600006,134.199997


In [51]:
np.array(file.iloc[:,0]).reshape(1,-1)

array([['TIF', 'CMG', 'DIS', ..., 'V', 'WMT', 'XOM']], dtype=object)

In [2]:

with open('process/process_file_progress_0.5.csv') as csv_file:
    reader = csv.reader(csv_file)
    data = list(reader)
data = np.array(data)
one_hot_encoder = preprocessing.OneHotEncoder()

processed_data = one_hot_encoder.fit_transform(data[:, 0:1]).toarray()
processed_data = np.append(processed_data, data[:, 1:6], 1)

processed_data = preprocessing.normalize(processed_data)
np.random.shuffle(processed_data)

In [61]:
processed_data[0,:]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [82]:
model = Sequential()
model.add(tf.keras.layers.Lambda(lambda x:tf.expand_dims(x, axis=-1), input_shape=[449]))
model.add(Conv1D(32, 5, activation="relu"))
model.add(MaxPooling1D(2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(Conv1D(64, 5, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(tf.keras.layers.Dropout(0.4))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))

model.compile(optimizer='sgd', metrics=['mse','mae'], loss='mse')
#model.summary(0)

In [56]:
np.expand_dims(processed_data, axis=-1).shape

NameError: name 'processed_data' is not defined

In [9]:
from sklearn.model_selection import train_test_split
X=processed_data[:,:-1]
y=processed_data[:,-1]
train_X, test_X, train_Y, test_Y=train_test_split(X,y,test_size=0.15)

In [10]:
train_X.shape

(18093, 449)

In [ ]:
history = model.fit(ds_train, epochs=50, verbose=2, validation_data = ds_test)

Epoch 1/50
